In [1]:
from sklearn import datasets
iris = datasets.load_iris()
X_iris, y_iris = iris.data, iris.target

In [2]:
print (X_iris.shape, y_iris.shape)
print (X_iris[0], y_iris[0])

(150, 4) (150,)
[ 5.1  3.5  1.4  0.2] 0
